<h1>VSS SEB Parsing for Ephemerides</h1>

In [18]:
import pandas as pd
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier


In [19]:
df = pd.DataFrame()

In [20]:
df = pd.read_excel('VSS SEB observed targets.xlsx', sheet_name='Data')

In [21]:
df.head()

,Star,Epoch (HJD),Epoch Error,Period (d),Period Error,O-C Diagram Notes,Recommended Fequency,Last Observed
0,Ant S,2.458519e+06,0.000287,0.648348,2.959964e-07,linear?,yearly,2022-03-08 15:49:44.602
1,Aps DW,2.439210e+06,0.000787,2.312944,9.672835e-08,linear?,yearly,2022-05-09 14:58:03.446
2,Aps II,2.451931e+06,0.149860,0.842229,1.695586e-05,eccentric + apsidal motion?,2 per year,2021-06-23 10:19:15.616
3,Aps MR,2.457159e+06,0.000462,0.527864,1.404435e-07,linear?,yearly,2023-07-26 11:58:57.014
4,Aps NT,2.457208e+06,0.000361,0.294762,5.285139e-08,downward parabola?,yearly,2024-05-23 13:12:24.365


In [22]:
df.columns

Index(['Star', 'Epoch (HJD)', 'Epoch Error', 'Period (d)', 'Period Error',
       'O-C Diagram Notes', 'Recommended Fequency', 'Last Observed'],
      dtype='object')

In [23]:
df['Const', 'VarID'] = df['Star'].str.split()

We parse the weirdness of the input file

In [101]:
objectsdf = pd.DataFrame()
for i in range(len(df)):
    details = df['Const', 'VarID'].iloc[i]
    if (len(details) == 2):
        object_name = details[1] +' '+ details[0]
    if (len(details) > 2):
        if (details[2].lower() == "secondary" or details[2].lower() == "primary"):
            object_name = details[1] +' '+ details[0]
        else:
            object_name = details[1]+details[2]# +' '+ details[0]

    result_table = Simbad.query_object(object_name)
    #print(result_table)
    if (result_table is None):
        print(f'{object_name}: failed to locate... looking at GCVS...')
        # if that is the case look at the GCVS
        gcvs_catalog = "B/gcvs"
        vizier = Vizier(columns=['RAJ2000,DEJ2000,magMax,VarType'])
        gcvs_result = vizier.query_object(object_name, catalog=gcvs_catalog)
        if gcvs_result:
            result_table=gcvs_result[0]
            print(' found in GCVS.')
        else:
            print(' could not find in GCVS.')
            continue
    objdf = result_table.to_pandas()
    for colname in df.columns[:-1]:
        #print(colname, df.iloc[i][colname])
        objdf[colname] = df.iloc[i][colname]
    objectsdf = pd.concat([objectsdf, objdf], ignore_index=True)
columns_to_drop = ['RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA','COO_ERR_ANGLE','COO_QUAL','COO_WAVELENGTH','COO_BIBCODE',	'SCRIPT_NUMBER_ID']
objectsdf = objectsdf.drop(columns=columns_to_drop)

NSV25657: failed to locate... looking at GCVS...
 found in GCVS.


ASASSN-VJ184530.00-593235.0: failed to locate... looking at GCVS...
 could not find in GCVS.


NSV1389: failed to locate... looking at GCVS...
 found in GCVS.


In [102]:
objectsdf['RA'] = objectsdf['RA'].str.replace(' ','', regex=True)
objectsdf['DEC'] = objectsdf['DEC'].str.replace(' ','', regex=True)
objectsdf['MAIN_ID'] = objectsdf['MAIN_ID'].str.replace('V* ','', regex=True)

In [109]:
objectsdf

,MAIN_ID,RA,DEC,Star,Epoch (HJD),Epoch Error,Period (d),Period Error,O-C Diagram Notes,Recommended Fequency,Last Observed,RAJ2000,DEJ2000,magMax,VarType
0,V*SAnt,093218.3864,-283739.968,Ant S,2.458519e+06,0.000287,0.648348,2.959964e-07,linear?,yearly,2022-03-08 15:49:44.602,NaN,NaN,NaN,NaN
1,V*DWAps,172330.0309,-675544.906,Aps DW,2.439210e+06,0.000787,2.312944,9.672835e-08,linear?,yearly,2022-05-09 14:58:03.446,NaN,NaN,NaN,NaN
2,V*IIAps,162741.9350,-745044.229,Aps II,2.451931e+06,0.149860,0.842229,1.695586e-05,eccentric + apsidal motion?,2 per year,2021-06-23 10:19:15.616,NaN,NaN,NaN,NaN
3,V*MRAps,170757.5281,-734335.400,Aps MR,2.457159e+06,0.000462,0.527864,1.404435e-07,linear?,yearly,2023-07-26 11:58:57.014,NaN,NaN,NaN,NaN
4,HD128575,144242.5763,-741841.568,Aps NT,2.457208e+06,0.000361,0.294762,5.285139e-08,downward parabola?,yearly,2024-05-23 13:12:24.365,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,V*V448Vel,082455.4977,-483334.971,Vel V0448,2.451868e+06,NaN,0.364876,NaN,only two ToMs,yearly,2021-03-04 14:09:17.856,NaN,NaN,NaN,NaN
209,BD-073477,124420.2386,-084016.846,Vir HW,2.452500e+06,0.000190,0.116720,3.396294e-09,linear?,yearly,2023-06-24 11:18:51.638,NaN,NaN,NaN,NaN
210,V*AEVol,084539.4033,-645928.043,Vol AE,2.452614e+06,0.000285,0.406615,1.940033e-08,linear?,yearly,2022-12-28 13:18:28.541,NaN,NaN,NaN,NaN
211,V*AFVol,085334.8637,-702807.633,Vol AF,2.452618e+06,0.000565,0.638935,6.027154e-08,linear?,yearly,2023-01-18 12:52:15.456,NaN,NaN,NaN,NaN


<hr>

Okay now time to format the dataframe to the format of the empherides cat

Here is a sample:

 CONS NAME             PS RATING RA2000   DE2000  M0           PER          MAX      PRIM     SEK      BRANCH LD   SD   GSC         TYPE         N10    N     
 And  RT               p  1      231110.1 +530133 51463.24     0.62892951   8.55     9.47     8.88     V      2.6       3998.02167  EA/RS        136    1576  


and it is formated Fortran style - columns have fixed width


cons = 6; name=17; PS=3, rating=7, RA2000=9, DE2000=8
M0=13, per=13, max=9, prim=9, sek=9, branch=7, ld=5, sd=5, GSC=12, type=13, N10=7, n=7

In [108]:
CONS = objectsdf['name'].str.split().str.get(1)
starname = objectsdf['name'].str.split().str.get(0)
ephem_df = pd.DataFrame()
ephem_df = ephem_df.assign(CONS=objectsdf['CONS'])
ephem_df = ephem_df.assign(NAME=objectsdf['starname'])
ephem_df['PS'] = "p"
ephem_df['PS'].loc[objectsdf['Star'].str.contains('Primary')] = "p"
ephem_df['PS'].loc[objectsdf['Star'].str.contains('Secondary')] = "s"

KeyError: 'CONS'

In [104]:
ephem['CONST'] = const
ephem['NAME'] = name
ephem['PS'] = ps
ephem['RATING'] = rating
ephem['RA2000'] = ra2000

In [105]:
# Convert DataFrame to a fixed-width formatted string
fixed_width_str = ephem.to_string(index=False, formatters=formatters)

In [106]:
fixed_width_str

'CONST               NAME  PS  RATING    RA2000\n  Ant  S                 p   10      93218.4  \n  Aps  DW                p   10      172330.0 \n  Aps  II                p   10      162741.9 \n  Aps  MR                p   10      170757.5 \n  Aps  NT                p   10      144242.6 \n  Aps  NX                p   10      145641.8 \n  Aps  TV                p   10      144549.7 \n  Aps  V0354             p   10      151528.6 \n  Aps  YY                p   10      150234.3 \n  Aqr  AO                p   10      221131.9 \n  Aqr  DX                p   10      220226.2 \n  Aqr  EE                p   10      223442.0 \n  Ara  FU                p   10      173332.3 \n  Ara  R                 p   10      163944.7 \n  Ara  V0489             p   10      171042.5 \n  Ara  V0491             p   10      171104.7 \n  Ara  V0535             p   10      173805.6 \n  Ara  V0610             p   10      165211.3 \n  Ara  V0870             p   10      180822.7 \n  Ara  V0878             p   10      16

In [134]:
# Save to a file
with open('fixed_width_file.txt', 'w') as file:
    file.write(fixed_width_str)
